<a href="https://colab.research.google.com/github/alisterpage/CHEM3580-Jupyter-Notebooks/blob/main/homework%201/forcefield_Uoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework - Forcefield Parameter Development: U$_\text{oop}$

For this homework exercise, we will use quantum chemistry to develop the forcefield parameters for the out-of-plane bending term ($U_\text{oop}(\chi$)) for two molecules - the methylene radical, and formaldehyde.


**Run the code cell below to setup the environment and software you need for this exercise.** (It may take a minute or so to run...)


In [1]:
#@title
!pip install pyscf geometric py3Dmol fortecubeview pythreejs -q 2>&1 >/dev/null
import pyscf
from pyscf import gto, scf, tools, lo, lib, dft
#from pyscf.geomopt.berny_solver import optimize
from pyscf.geomopt.geometric_solver import optimize
from geometric.molecule import Molecule
import py3Dmol
import fortecubeview
from google.colab import output
output.enable_custom_widget_manager()
import numpy as np
from __future__ import print_function
import sys

### Part 1: Methylene Radical

The code cells below will optimise the geometry of the methylene radical with the H-H-H-C dihedral angle $\chi$ frozen at the value you specify. The code will use density functional theory (B3LYP/6-31G(d)), which you have used previously in CHEM2410.

Use this code to calculate the constrained energy of the methylene radical as a function of $\chi$, for several different $\chi$ values (consider a range of 0 - 50 degrees). Ensure that the optimised χ value (reported at the end of the calculation) is approximately the same as the value you fixed it at, and inspect the optimised structure to ensure that it is reasonable. Repeat the calculation as many times as you think necessary to get a reliable correlation between the optimised energy and the dihedral angle $\chi$.



**Enter the value of the dihedral angle you want in the text box below.**

In [2]:
#@title 🎯🎯🎯
import ipywidgets as widgets

dihedral_textbox = widgets.FloatText(
    value=0.0,
    description='Dihedral Angle (degrees):',
    disabled=False
)

display(dihedral_textbox)

# Get the value entered by the user in the widget and store it as a float variable called dihedral


FloatText(value=0.0, description='Dihedral Angle (degrees):')

**Run the next code cell to do the B3LYP/6-31G(d) geometry optimisation**

(You can ignore the output from this cell - the cell below summarises the key results for you)

In [3]:
#@title 🎯🎯🎯
#define molecule
initial_dihedral = dihedral_textbox.value
mol = gto.Mole()
mol.atom = '''
    H
    H 1 1.8
    H 2 1.8 1 60.0
    C 1 1.1 2 35.2 3 {}
'''.format(initial_dihedral)

#define model chemistry & calculation
mol.basis = '6-31g(d)'
mol.spin=1;
mol.build();
mf = dft.ROKS(mol)
mf.xc = 'b3lyp'
mf = mf.newton()
mf.kernel()

# Optimize the geometry
text = "$freeze\ndihedral 1 2 3 4\n"
with open("constraints.txt", "w") as f:
    f.write(text)
params = {"constraints": "constraints.txt",}
mol_eq = optimize(mf, **params)

#print(mol_eq.atom_coords())

# Get the optimized energy
mf = dft.ROKS(mol_eq)
mf.xc = 'b3lyp'
mf = mf.newton()
mf.kernel()

# Create a temporary XYZ file with the optimized coordinates
symbols = [atom[0] for atom in mol_eq.atom];
coords = mol_eq.atom_coords();
with open("temp.xyz", "w") as f:
    f.write("%d\n" % len(symbols))
    f.write("Generated by PySCF and Geometric\n")
    for i in range(len(symbols)):
        f.write("%s %f %f %f\n" % (symbols[i], coords[i,0], coords[i,1], coords[i,2]))

# Convert the PySCF object to a `Molecule` object using the XYZ file and store dihedral angle.
mol_geom = Molecule("temp.xyz");
dihedral=Molecule.measure_dihedrals(mol_geom,0,1,2,3);

/usr/local/lib/python3.11/dist-packages/pyscf/dft/libxc.py:512: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


converged SCF energy = -39.8108667262493


geometric-optimize called with the following command line:
/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-137e1141-68d4-488c-a3af-b67a057fd206.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********    


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   H   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   1.800000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.900000   0.000000   1.558846    0.000000  0.000000  0.000000
   C   0.898859   0.000000   0.634076    0.000000  0.000000  0.000000
converged SCF energy = -39.8108667262519
--------------- SecondOrderROKS_Scanner gradients ---------------
         x                y                z
0 H    -0.0020232548     0.0000000000    -0.0154010438
1 H     0.0030249963     0.0000000000    -0.0159131107
2 H    -0.0000707577     0.0000000000    -0.1789145881
3 C    -0.0009310381    -0.0000000000     0.2102307022
----------------------------------------------
cycle 1: E = -39.8108667263  dE = -39.8109  norm(grad) = 0.276969


Step    0 : Gradient = 1.385e-01/2.102e-01 (rms/max) Energy = -39.8108667263
Hessian Eigenvalues: 3.06588e-02 5.00000e-02 5.00000e-02 ... 3.34767e-01 3.36821e-01 6.37970e-01


...

Step    3 : Displace = 2.336e-02/2.798e-02 (rms/max) Trust = 1.414e-01 (=) Grad_T = 2.201e-04/2.725e-04 (rms/max) E (change) = -39.8359973320 (-4.035e-04) Quality = 0.963
Hessian Eigenvalues: 3.06588e-02 5.00000e-02 5.00000e-02 ... 3.35778e-01 3.52357e-01 5.78693e-01



Geometry optimization cycle 5
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   H  -0.037669   0.000000   0.007075    0.000816 -0.000000 -0.000377
   H   1.837085   0.000000   0.007126   -0.000808 -0.000000 -0.000433
   H   0.899755   0.000000   1.630646   -0.000050 -0.000000  0.000418
   C   0.899690   0.000000   0.548166    0.000042 -0.000000  0.000486
converged SCF energy = -39.8359978437722
--------------- SecondOrderROKS_Scanner gradients ---------------
         x                y                z
0 H     0.0000052844     0.0000000000     0.0000112926
1 H    -0.0000003520    -0.0000000000     0.0000181519
2 H     0.0000084083     0.0000000000     0.0000947792
3 C    -0.0000133377    -0.0000000000    -0.0001264983
----------------------------------------------
cycle 5: E = -39.8359978438  dE = -5.11755e-07  norm(grad) = 0.00016037


Step    4 : Displace = 7.176e-04/9.292e-04 (rms/max) Trust = 2.000e-01 (+) Grad_T = 8.019e-05/1.272e-04 (rms/max) E (change) = -39.8359978438 (-5.118e-07) Quality = 1.055
Hessian Eigenvalues: 3.06588e-02 5.00000e-02 5.00000e-02 ... 3.35778e-01 3.52357e-01 5.78693e-01
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 20.470 seconds


converged SCF energy = -39.8359978437771


[(0, 1), (1, 2), (2, 3)]Measuring dihedral angle for four atoms that aren't bonded.  Hope you know what you're doing!

**Run the next code cell to summarise the calculation and show the geometry of the molecule**

In [4]:
#@title 🎯🎯🎯
#print summary
print('Initial dihedral before geometry optimisation:', initial_dihedral,"degrees")
print('Final dihedral after geometry optimisation:', abs(dihedral[0]), 'degrees')
print('Final energy after geometry optimisation:', mf.e_tot,"Hartree")

# get the atomic positions & write PDB file
atom_pos = mol.atom_coords()
atom_symbol = mol._atom

with open('mol.xyz', 'w') as f:
    f.write(f'{len(mol._atom)}\n')
    f.write('opt geom\n')
    for i, (symbol, pos) in enumerate(zip(atom_symbol, atom_pos)):
        f.write(f'{symbol[0]}  {pos[0]:>7.3f}{pos[1]:>7.3f}{pos[2]:>7.3f}\n')


view = py3Dmol.view()
view.addModel(open('mol.xyz', 'r').read(),'xyz')
view.setBackgroundColor('#CFB691')
view.setStyle({'sphere': {'radius': 0.5}})
view.zoomTo()
view.show()


Initial dihedral before geometry optimisation: 0.0 degrees
Final dihedral after geometry optimisation: 0.0 degrees
Final energy after geometry optimisation: -39.835997843777086 Hartree


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Part 2: Formaldehyde

The code cells below will optimise the geometry of formaldehyde with the O-H-H-C dihedral angle $\chi$ frozen at the value you specify. The code will use density functional theory (B3LYP/6-31G(d)), which you have used previously in CHEM2410. _This is the same level of theory used for methylene, above_.

Use this code to calculate the constrained energy of formaldehyde as a function of $\chi$, for several different $\chi$ values (consider a range of 0 - 50 degrees). Ensure that the optimised χ value (reported at the end of the calculation) is approximately the same as the value you fixed it at, and inspect the optimised structure to ensure that it is reasonable. Repeat the calculation as many times as you think necessary to get a reliable correlation between the optimised energy and the dihedral angle $\chi$.



**Enter the value of the dihedral angle you want in the text box below.**

In [5]:
#@title  🎯🎯🎯
import ipywidgets as widgets

dihedral_textbox = widgets.FloatText(
    value=0.0,
    description='Dihedral Angle (degrees):',
    disabled=False
)

display(dihedral_textbox)

# Get the value entered by the user in the widget and store it as a float variable called dihedral


FloatText(value=0.0, description='Dihedral Angle (degrees):')

**Run the next code cell to do the B3LYP/6-31G(d) geometry optimisation**

(You can ignore the output from this cell - the cell below summarises the key results for you)

In [6]:
#@title  🎯🎯🎯
#define molecule
initial_dihedral = dihedral_textbox.value
mol = gto.Mole()
mol.atom = '''
    O
    H 1 2.03
    H 2 1.87 1 62.0
    C 1 1.2 2 27.5 3 {}
'''.format(initial_dihedral)

#define model chemistry & calculation
mol.basis = '6-31g(d)'
mol.spin=0;
mol.build();
mf = dft.RKS(mol)
mf.xc = 'b3lyp'
mf = mf.newton()
mf.kernel()

# Optimize the geometry
text = "$freeze\ndihedral 1 2 3 4\n"
with open("constraints.txt", "w") as f:
    f.write(text)
params = {"constraints": "constraints.txt",}
mol_eq = optimize(mf, **params)

#print(mol_eq.atom_coords())

# Get the optimized energy
mf = dft.RKS(mol_eq)
mf.xc = 'b3lyp'
mf = mf.newton()
mf.kernel()

# Create a temporary XYZ file with the optimized coordinates
symbols = [atom[0] for atom in mol_eq.atom];
coords = mol_eq.atom_coords();
with open("temp.xyz", "w") as f:
    f.write("%d\n" % len(symbols))
    f.write("Generated by PySCF and Geometric\n")
    for i in range(len(symbols)):
        f.write("%s %f %f %f\n" % (symbols[i], coords[i,0], coords[i,1], coords[i,2]))

# Convert the PySCF object to a `Molecule` object using the XYZ file and store dihedral angle.
mol_geom = Molecule("temp.xyz");
dihedral=Molecule.measure_dihedrals(mol_geom,0,1,2,3);

converged SCF energy = -114.498075600104


geometric-optimize called with the following command line:
/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-137e1141-68d4-488c-a3af-b67a057fd206.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********    


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   2.030000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   1.152088   0.000000   1.651112    0.000000  0.000000  0.000000
   C   1.064413   0.000000   0.554098    0.000000  0.000000  0.000000
converged SCF energy = -114.498075600096
--------------- SecondOrderRKS_Scanner gradients ---------------
         x                y                z
0 O     0.0100598509    -0.0000000000     0.0063694309
1 H     0.0010096455     0.0000000000    -0.0002769819
2 H    -0.0013259256    -0.0000000000    -0.0064134995
3 C    -0.0097126876    -0.0000000000     0.0003542970
----------------------------------------------
cycle 1: E = -114.4980756  dE = -114.498  norm(grad) = 0.0167398


Step    0 : Gradient = 8.370e-03/1.191e-02 (rms/max) Energy = -114.4980756001
Hessian Eigenvalues: 3.47347e-02 5.00000e-02 5.00000e-02 ... 3.22556e-01 3.36256e-01 1.04823e+00



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000076   0.000000  -0.011488    0.000076  0.000000 -0.011488
   H   2.024159  -0.000000  -0.004052   -0.005841 -0.000000 -0.004052
   H   1.156889   0.000000   1.663105    0.004801  0.000000  0.011993
   C   1.064071   0.000000   0.556237   -0.000342  0.000000  0.002139
converged SCF energy = -114.498192025351
--------------- SecondOrderRKS_Scanner gradients ---------------
         x                y                z
0 O     0.0016953430    -0.0000000000    -0.0011204567
1 H    -0.0002615003    -0.0000000000    -0.0014615303
2 H     0.0005882437    -0.0000000000     0.0001996024
3 C    -0.0019991152     0.0000000000     0.0024176532
----------------------------------------------
cycle 2: E = -114.498192025  dE = -0.000116425  norm(grad) = 0.00406958


...

Step    4 : Displace = 7.454e-04/9.273e-04 (rms/max) Trust = 6.659e-03 (+) Grad_T = 8.612e-05/1.283e-04 (rms/max) E (change) = -114.4982166611 (-1.100e-06) Quality = 1.026
Hessian Eigenvalues: 3.47347e-02 4.74148e-02 5.00000e-02 ... 3.24918e-01 5.40422e-01 1.03230e+00



Geometry optimization cycle 6
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.003626   0.000000  -0.009997   -0.000150  0.000000 -0.000663
   H   2.027023  -0.000000  -0.001540   -0.000195  0.000000 -0.000163
   H   1.153747   0.000000   1.658697   -0.000683  0.000000 -0.000382
   C   1.064066   0.000000   0.551739   -0.000269  0.000000 -0.000339
converged SCF energy = -114.49821667398
--------------- SecondOrderRKS_Scanner gradients ---------------
         x                y                z
0 O     0.0000323793    -0.0000000000     0.0000165106
1 H     0.0000042352    -0.0000000000    -0.0000055003
2 H     0.0000166605    -0.0000000000     0.0000056353
3 C    -0.0000252014    -0.0000000000     0.0000172072
----------------------------------------------
cycle 6: E = -114.498216674  dE = -1.28494e-08  norm(grad) = 5.10856e-05


Step    5 : Displace = 5.379e-05/7.296e-05 (rms/max) Trust = 9.418e-03 (+) Grad_T = 2.554e-05/3.635e-05 (rms/max) E (change) = -114.4982166740 (-1.285e-08) Quality = 0.351
Hessian Eigenvalues: 3.47347e-02 4.74148e-02 5.00000e-02 ... 3.24918e-01 5.40422e-01 1.03230e+00
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 13.099 seconds


converged SCF energy = -114.498216674019


[(0, 1), (1, 2), (2, 3)]Measuring dihedral angle for four atoms that aren't bonded.  Hope you know what you're doing!

**Run the next code cell to summarise the calculation and show the geometry of the molecule**

**NOTE: The 'Final dihedral' value is the one you need to use in determining $k_c$.**

In [7]:
#@title 🎯🎯🎯
#print summary
print('Initial dihedral before geometry optimisation:', initial_dihedral,"degrees")
print('Final dihedral after geometry optimisation:', abs(dihedral[0]), 'degrees')
print('Final energy after geometry optimisation:', mf.e_tot,"Hartree")

# get the atomic positions & write PDB file
atom_pos = mol.atom_coords()
atom_symbol = mol._atom

with open('mol.xyz', 'w') as f:
    f.write(f'{len(mol._atom)}\n')
    f.write('opt geom\n')
    for i, (symbol, pos) in enumerate(zip(atom_symbol, atom_pos)):
        f.write(f'{symbol[0]}  {pos[0]:>7.3f}{pos[1]:>7.3f}{pos[2]:>7.3f}\n')


view = py3Dmol.view()
view.addModel(open('mol.xyz', 'r').read(),'xyz')
view.setBackgroundColor('#CFB691')
view.setStyle({'sphere': {'radius': 0.5}})
view.zoomTo()
view.show()


Initial dihedral before geometry optimisation: 0.0 degrees
Final dihedral after geometry optimisation: 0.0 degrees
Final energy after geometry optimisation: -114.49821667401876 Hartree


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Questions

1. Use your data to estimate the out-of-plane bending force constant $k^c$ for the out-of-plane bending potentials in the methylene radical and formaldeyde. (It is easiest to do this analysis in Microsoft Excel.)
1. Both of these molecules have equivalent structure and a central sp$^2$-hybridised carbon atom. Why do you think the $k^c$ values for these two potentials are so different? Explain your answer in terms of the electronic structure and bonding in these molecules.

**Submit your homework as a single MS excel file. You will be assessed on (1) the accuracy of your force constant value, and its $\pm$ margin of error, and (2) your answer to question 2, above.**